In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/11120/DS/hw/hw2')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_test['Weather'] = np.zeros((len(df_test),))
train_end_idx = len(df)
df = pd.concat([df, df_test], sort=False)

#encode categories
from sklearn.preprocessing import LabelEncoder
df['Loc'] = LabelEncoder().fit_transform(df['Loc'])
df['WindDir'] = LabelEncoder().fit_transform(df['WindDir'])

df = df.drop(columns = [col for col in df.columns if df[col].dtype == object])

#impute missing data
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
imp = IterativeImputer(missing_values=np.nan, initial_strategy='median')
imputed_data = imp.fit_transform(df.values)
df_impute = pd.DataFrame(imputed_data)
df_impute.columns = df.columns

X = df_impute.drop(columns = ['Weather']).values[:train_end_idx, :]
y = df_impute['Weather'].values[:train_end_idx]
X_ans = df_impute.drop(columns = ['Weather']).values[train_end_idx:, :]

/usr/local/lib/python3.9/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [ ]:
X_train0, X_test, y_train0, y_test = train_test_split(X, y, test_size=0.001)

#oversampling
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
X_train, y_train = RandomOverSampler(random_state=0).fit_resample(X_train0, y_train0)

from imblearn.over_sampling import SMOTE, ADASYN
#X_train, y_train = ADASYN(random_state=0).fit_resample(X_train0, y_train0)

from imblearn.combine import SMOTETomek, SMOTEENN
#X_train, y_train = SMOTETomek(random_state=0).fit_resample(X_train0, y_train0)

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
cf = RandomForestClassifier(n_estimators=101, max_depth=17, max_samples=0.5)

from imblearn.ensemble import BalancedRandomForestClassifier
#cf = BalancedRandomForestClassifier(n_estimators=101, max_depth=17, max_samples=0.5)
model = cf.fit(X_train, y_train)

#prediction
y_pred = model.predict(X_train)
print(f'accuracy: {accuracy_score(y_train, y_pred):.6f}')
print(f'f1_score: {f1_score(y_train, y_pred):.6f}')
print('-'*6)
y_pred = model.predict(X_train0)
print(f'accuracy: {accuracy_score(y_train0, y_pred):.6f}')
print(f'f1_score: {f1_score(y_train0, y_pred):.6f}')
print('-'*6)
y_pred = model.predict(X_test)
print(f'accuracy: {accuracy_score(y_test, y_pred):.6f}')
print(f'f1_score: {f1_score(y_test, y_pred):.6f}')

ans_pred = model.predict(X_ans)
df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
df_sap.to_csv('myAns.csv',  index_label = 'Id')

accuracy: 0.983270
f1_score: 0.983546
------
accuracy: 0.970047
f1_score: 0.874955
------
accuracy: 1.000000
f1_score: 0.000000


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
